# Imports


In [54]:
import numpy as np
import os

# Access functions

In [37]:
def retrieve_time(path):
    """
    Input: inner path to a result (results/logs/path/job.out)
    Output: a float or an error
    """
    f = open("results/logs/" + path + "/job.out", "r")
    return float(f.readlines()[1][6:])

In [72]:
def retrieve_settings(path):
    """
    Input: inner path to a result (results/logs/path/job.out)
    Output: a numpy array of settings
    """
    options = path.split("/")
    settings = []
    for set in options:
        if not str.isalpha(set.split("_")[-1]):
            settings.append(set.split("_")[-1])
    return np.array(settings, dtype=np.float64)

In [52]:
def create_dataset(path = "", dataset = np.array([], dtype=np.float64)):
    if os.listdir(path) == []:
        return dataset
    elif "job.out" in os.listdir(path):
        try:
            time = retrieve_time(path)
            settings = retrieve_settings(path)
            return np.append(dataset, np.append(settings, np.float64(time)))
        except:
            return dataset
    else:
        for dir in os.listdir(path):
            dataset = create_dataset("results/logs/" + path, dataset)

    return dataset

# Tests


In [73]:
retrieve_settings("pthreads/threads_2/seq_length_10/patterns_10/mean_path_length_6")

array([ 2., 10., 10.,  6.])

In [51]:
create_dataset("pthreads/threads_2/seq_length_10/patterns_10/mean_path_length_6")

['job.out', 'job.log', 'job.err']

In [39]:
retrieve_time("pthreads/threads_2/seq_length_10/patterns_10/mean_path_length_4")

0.006373